## Tokens
##### Keywords: if, else, elif, while, for, in, def, return, True, False, None
##### Operators: +, -, *, /, %, **, =, ==, !=, <, >, <=, >=, and, or, not
##### Delimiters: (, ), {, }, [, ], ',' , '.', ':', ';'
##### Literals: INTEGER, FLOAT, STRING
##### Other: IDENTIFIER, INDENT, DEDENT, NEWLINE, EOF

## Token and Error Class

In [2]:
import re
from typing import Generator, List, Optional, Tuple

class Token:
    def __init__(self, type: str, value: str, line: int, column: int):
        self.type = type
        self.value = value
        self.line = line
        self.column = column
    
    def __str__(self) -> str:
        return f"Token({self.type}, '{self.value}', line={self.line}, col={self.column})"
    
    def __repr__(self) -> str:
        return self.__str__()

class Error:
    def __init__(self, message: str, line: int, column: int):
        self.message = message
        self.line = line
        self.column = column
    
    def __str__(self) -> str:
        return f"Error: {self.message} at line {self.line}, column {self.column}"


## Lexer

In [4]:
class Lexer:
    # Defining regular expressions for token
    TOKEN_SPECS = [
        ('COMMENT', r'#.*'),                                  # Comments
        ('STRING', r'\"([^\\\"]|\\.)*\"|\'([^\\\']|\\.)*\''), # String literals
        ('FLOAT', r'\d+\.\d+'),                               # Float literals
        ('INTEGER', r'\d+'),                                  # Integer literals
        ('KEYWORD', r'(if|else|elif|while|for|in|def|return|True|False|None)\b'),  # Keywords
        ('IDENTIFIER', r'[a-zA-Z_]\w*'),                      # Identifiers
        # Operators (multi-character ones first)
        ('OP_EQ', r'=='),
        ('OP_NE', r'!='),
        ('OP_LE', r'<='),
        ('OP_GE', r'>='),
        ('OP_ASSIGN', r'='),
        ('OP_PLUS', r'\+'),
        ('OP_MINUS', r'-'),
        ('OP_MULT', r'\*\*|\/\/|\*|\/'),  # ** (power), // (floor div), * (mult), / (div)
        ('OP_MOD', r'%'),
        ('OP_LT', r'<'),
        ('OP_GT', r'>'),
        # Delimiters
        ('LPAREN', r'\('),
        ('RPAREN', r'\)'),
        ('LBRACKET', r'\['),
        ('RBRACKET', r'\]'),
        ('LBRACE', r'\{'),
        ('RBRACE', r'\}'),
        ('COMMA', r','),
        ('DOT', r'\.'),
        ('COLON', r':'),
        ('SEMICOLON', r';'),
        ('NEWLINE', r'\n'),
        ('WHITESPACE', r'[ \t]+'),                            # Whitespace
        # INDENT and DEDENT tokens are not matched by regex but generated based on whitespace
    ]
    
    def __init__(self, source_code: str):
        self.source_code = source_code
        self.tokens = []
        self.errors = []
        self.line = 1
        self.column = 1
        self.indent_levels = [0]  # Start with indent level 0
    
    def tokenize(self) -> Generator[Token, None, None]:
        """Tokenize the source code and yield tokens."""
        # Process the source code line by line to handle indentation properly
        lines = self.source_code.split('\n')
        
        for line_num, line in enumerate(lines):
            self.line = line_num + 1
            self.column = 1
            
            #calculating the amount of leading whitespace (indentation) at the beginning of the current line
            if line.strip():  # Non-empty line      #strip() is equivalent of trim() in JS
                indent_size = len(line) - len(line.lstrip())    #lstrip() will remove leading spaces in the line i.e. remove any indentation; and the difference in length tells you how much whitespace there was
                indent_tokens = self._handle_indentation(indent_size)
                for token in indent_tokens:
                    yield token
            else:   # Skip empty lines but still count them for line numbers
                yield Token('NEWLINE', '\\n', self.line, self.column)
                continue
            
            # Process the rest of the line
            i = indent_size if 'indent_size' in locals() else 0
            line_content = line
            
            while i < len(line_content):    #traversing a line character by character
                match = None
                
                ## Skip spaces and tabs (already handled indentation)
                ##if line_content[i].isspace():   #isspace() checks if the entire line is whitespace or not; here it checks whether a single character is whitespace or not since its applied on line_content[i]
                ##    i += 1
                ##    self.column += 1
                ##    continue
                
                # Try to match each token pattern
                for token_type, pattern in self.TOKEN_SPECS:
                    regex = re.compile(pattern) #compiling the pattern into a regex so that we can use it to find tokens in the line
                    match = regex.match(line_content, i)    #match() will check if the "starting" of the string matches the given regex or not, line_content gives the line whose starting it has to compare with the regex with i telling from which point in the line to assume as thhe starting of the line
                    #if a match is found, 'match' will contain an object with info about the found match (like start and end, its value and so on), otherwise it'll contain "none"
                    
                    if match:
                        value = match.group(0)  #group will give the value of the match object
                        
                        if token_type == 'WHITESPACE':
                            #whitespace is not yielded becuz parser receives stream of tokens without space and doesnt care about space
                            # Just update column and continue
                            self.column += len(value)
                            i += len(value)
                            continue
                        elif token_type == 'COMMENT':
                            # Ignore comments
                            i += len(value)
                            self.column += len(value)
                            break   # Break out of the current character processing loop after handling comments
                        else:
                            # For all other tokens
                            token = Token(token_type, value, self.line, self.column)
                            yield token
                        
                        i += len(value)
                        self.column += len(value)
                        break
                
                if not match:
                    # No token matched, raise an error
                    error_msg = f"Invalid character: '{line_content[i]}'"
                    error = Error(error_msg, self.line, self.column)
                    self.errors.append(error)
                    print(error)  # Print the error but continue
                    i += 1
                    self.column += 1
            
            # Add NEWLINE at the end of each line except the last one
            if line_num < len(lines) - 1 or self.source_code.endswith('\n'):
                yield Token('NEWLINE', '\\n', self.line, self.column)
        
        # Output any pending dedents at the end of the file
        #end of file means all indented blocks have been dedented so pop all elements of the indent_levels stack
        while len(self.indent_levels) > 1:
            self.indent_levels.pop()    
            yield Token('DEDENT', '', self.line, self.column)
        
        # End of file token
        yield Token('EOF', '', self.line, self.column)
    
    def _handle_indentation(self, indent_size: int) -> List[Token]:
        """Handle Python's indentation-based block structure. Returns a list of INDENT or DEDENT tokens as needed."""
        tokens = []
        previous_line_indent = self.indent_levels[-1]
        
        if indent_size > previous_line_indent:
            # This is an indentation (start of a new block)
            self.indent_levels.append(indent_size)  #when a new block is recognized through indentation its indentation level is pushed in the stack so that w ecan later check whether that block was dedented properly or not
            tokens.append(Token('INDENT', ' ' * (indent_size - previous_line_indent), self.line, 1))
        
        elif indent_size < previous_line_indent:
            # This is a dedentation (end of one or more blocks)
            while self.indent_levels and indent_size < self.indent_levels[-1]: # ensuring that the latest code block is being dedented
                self.indent_levels.pop()
                tokens.append(Token('DEDENT', '', self.line, 1))
            
            if indent_size != self.indent_levels[-1]:
                # Invalid indentation
                error_msg = f"Inconsistent indentation"
                error = Error(error_msg, self.line, 1)
                self.errors.append(error)
                print(error)  # Print the error but continue
                
                
                # and what about handling indent_size == self.indent_levels[-1]
        return tokens

## Test Script for Lexer

In [ ]:
def test_lexer(source_code, test_name=""):
    print(f"\n=== Testing {test_name} ===")
    print(f"Source code:\n{source_code}")
    print("\nTokens:")
    
    lexer = Lexer(source_code)
    token_count = 0
    
    try:
        for token in lexer.tokenize():
            print(token)
            token_count += 1
        
        print(f"\nTotal tokens: {token_count}")
        if lexer.errors:
            print(f"\nErrors ({len(lexer.errors)}):")
            for error in lexer.errors:
                print(f"  {error}")
    except Exception as e:
        print(f"Exception occurred: {str(e)}")
        import traceback
        traceback.print_exc()

# Test cases
if __name__ == "__main__":
    # Test 1: Basic variable assignment
    test_lexer("x = 10", "Basic Assignment")

    # Test 2: Simple function definition
    test_lexer("""def add(a, b):
    return a + b""", "Function Definition")

    # Test 3: If statement with proper indentation
    test_lexer("""if x > 10:
    print("Greater than 10")
else:
    print("Less than or equal to 10")""", "If Statement")

    # Test 4: Various literals and operators
    test_lexer("""# Testing literals
x = 42
y = 3.14
name = "John"
flag = True
result = x + y * 2
equal = x == y""", "Literals and Operators")

    # Test 5: Loops
    test_lexer("""# Testing loops
for i in range(10):
    if i % 2 == 0:
        print("Even")
    else:
        print("Odd")

# While loop
count = 0
while count < 5:
    count += 1
""", "Loops")

    # Test 6: Test error handling
    test_lexer("x = 10 @", "Error Handling")

    # Test 7: Test inconsistent indentation
    test_lexer("""def test():
    x = 1
   y = 2  # Inconsistent indentation
""", "Inconsistent Indentation")

    # Interactive testing option
    do_interactive = input("\nDo you want to run an interactive test? (y/n): ")
    if do_interactive.lower() == 'y':
        print("\n=== Interactive Lexer Test ===")
        print("Enter Python code (type 'exit()' on a new line to finish):")
        
        lines = []
        while True:
            line = input("> ")
            if line.strip() == "exit()":
                break
            lines.append(line)
        
        source_code = "\n".join(lines)
        test_lexer(source_code, "Interactive Input")


## ***ABSTRACT SYNTAX TREE***

In [5]:
#This file is of abstract syntax tree (AST) node definitions.

from dataclasses import dataclass
from typing import List, Optional, Union

# Base class for all AST nodes ASTNode is the base (parent) class for all other AST node types.
#It doesn’t do anything itself, but helps us group all node types together under one type.

class ASTNode:
    pass

# Root node; body is a list of things in the program like function definitions, statements, etc.
@dataclass
class Program(ASTNode):
    body: List[ASTNode]

# Statements
@dataclass
class FunctionDef(ASTNode):
    name: str
    params: List[str]
    body: List[ASTNode]

@dataclass
class IfStatement(ASTNode):
    condition: ASTNode
    then_branch: List[ASTNode]
    elif_branches: List[tuple[ASTNode, List[ASTNode]]]
    else_branch: Optional[List[ASTNode]]

@dataclass
class WhileLoop(ASTNode):
    condition: ASTNode
    body: List[ASTNode]

@dataclass
class ForLoop(ASTNode):
    var: str
    iterable: ASTNode
    body: List[ASTNode]

@dataclass
class Return(ASTNode):
    value: Optional[ASTNode]

@dataclass
class Assignment(ASTNode):
    target: str
    value: ASTNode

@dataclass
class ExpressionStatement(ASTNode):
    expression: ASTNode

# Expressions
@dataclass
class BinaryOp(ASTNode):
    left: ASTNode
    operator: str
    right: ASTNode

@dataclass
class UnaryOp(ASTNode):
    operator: str
    operand: ASTNode

@dataclass
class Call(ASTNode):
    func: ASTNode
    args: List[ASTNode]

@dataclass
class Identifier(ASTNode):
    name: str

@dataclass
class Literal(ASTNode):
    value: Union[str, int, float, bool, None]


## ***AST VISUALIZER***

In [6]:
from graphviz import Digraph
from typing import Union, List

def print_ast(node: ASTNode, indent: str = ''):
    """Print the AST in a tree-like structure in the console."""
    if isinstance(node, Program):
        print(indent + "Program")
        for stmt in node.body:
            print_ast(stmt, indent + "  ")
    elif isinstance(node, FunctionDef):
        print(indent + f"FunctionDef({node.name})")
        for stmt in node.body:
            print_ast(stmt, indent + "  ")
    elif isinstance(node, IfStatement):
        print(indent + "IfStatement")
        print(indent + "  Condition:")
        print_ast(node.condition, indent + "    ")
        print(indent + "  Then:")
        for stmt in node.then_branch:
            print_ast(stmt, indent + "    ")
        for cond, branch in node.elif_branches:
            print(indent + "  Elif:")
            print_ast(cond, indent + "    ")
            for stmt in branch:
                print_ast(stmt, indent + "      ")
        if node.else_branch:
            print(indent + "  Else:")
            for stmt in node.else_branch:
                print_ast(stmt, indent + "    ")
    elif isinstance(node, WhileLoop):
        print(indent + "WhileLoop")
        print(indent + "  Condition:")
        print_ast(node.condition, indent + "    ")
        print(indent + "  Body:")
        for stmt in node.body:
            print_ast(stmt, indent + "    ")
    elif isinstance(node, ForLoop):
        print(indent + f"ForLoop(var={node.var})")
        print(indent + "  Iterable:")
        print_ast(node.iterable, indent + "    ")
        print(indent + "  Body:")
        for stmt in node.body:
            print_ast(stmt, indent + "    ")
    elif isinstance(node, Return):
        print(indent + "Return")
        if node.value:
            print_ast(node.value, indent + "  ")
    elif isinstance(node, Assignment):
        print(indent + f"Assignment(target={node.target})")
        print_ast(node.value, indent + "  ")
    elif isinstance(node, ExpressionStatement):
        print(indent + "ExpressionStatement")
        print_ast(node.expression, indent + "  ")
    elif isinstance(node, BinaryOp):
        print(indent + f"BinaryOp({node.operator})")
        print_ast(node.left, indent + "  ")
        print_ast(node.right, indent + "  ")
    elif isinstance(node, UnaryOp):
        print(indent + f"UnaryOp({node.operator})")
        print_ast(node.operand, indent + "  ")
    elif isinstance(node, Call):
        print(indent + "Call")
        print_ast(node.func, indent + "  ")
        for arg in node.args:
            print_ast(arg, indent + "    ")
    elif isinstance(node, Identifier):
        print(indent + f"Identifier({node.name})")
    elif isinstance(node, Literal):
        print(indent + f"Literal({repr(node.value)})")
    else:
        print(indent + f"UnknownNode({node})")

# Optional: Graphviz visualizer
def visualize_ast(node: ASTNode) -> Digraph:
    dot = Digraph()
    _build_graph(node, dot)
    return dot

def _build_graph(node: ASTNode, dot: Digraph, parent: str = None, counter=[0]):
    node_id = f"node{counter[0]}"
    counter[0] += 1

    label = type(node).__name__
    if isinstance(node, Identifier):
        label += f"({node.name})"
    elif isinstance(node, Literal):
        label += f"({repr(node.value)})"
    elif isinstance(node, Assignment):
        label += f"({node.target})"
    elif isinstance(node, BinaryOp):
        label += f"({node.operator})"
    elif isinstance(node, UnaryOp):
        label += f"({node.operator})"
    elif isinstance(node, FunctionDef):
        label += f"({node.name})"
    elif isinstance(node, ForLoop):
        label += f"({node.var})"

    dot.node(node_id, label)

    if parent:
        dot.edge(parent, node_id)

    for field in getattr(node, '__dataclass_fields__', {}):
        child = getattr(node, field)
        if isinstance(child, ASTNode):
            _build_graph(child, dot, node_id, counter)
        elif isinstance(child, list):
            for item in child:
                if isinstance(item, ASTNode):
                    _build_graph(item, dot, node_id, counter)
        elif isinstance(child, tuple):
            for item in child:
                if isinstance(item, ASTNode):
                    _build_graph(item, dot, node_id, counter)

    return dot


## ***PARSER***

In [7]:
# parser.py

from treeNodes import *  # Import AST node classes like Program, Identifier, etc.

# Custom error class for parser errors
class ParserError(Exception):
    pass

# Parser class converts tokens into an AST
class Parser:
    def __init__(self, lexer: Lexer):
        self.tokens = list(lexer.tokenize())  # Convert lexer generator into list of tokens
        self.pos = 0  # Current position in token list
        self.current_token = self.tokens[self.pos]  # Currently looked-at token

    def error(self, msg: str):
        # Raise a ParserError with current token's position
        raise ParserError(f"{msg} at line {self.current_token.line}, column {self.current_token.column}")
    
    def skip_newlines(self):
        # Ignore NEWLINE tokens before/after blocks or statements
        while self.current_token.type == 'NEWLINE':
            self.advance()

    def advance(self):
        # Move to the next token
        self.pos += 1
        if self.pos < len(self.tokens):
            self.current_token = self.tokens[self.pos]

    def expect(self, token_type: str):
        # Ensure the current token is of the expected type, or throw error
        if self.current_token.type == token_type:
            self.advance()
        else:
            self.error(f"Expected token {token_type}, got {self.current_token.type}")

    def match(self, token_type: str):
        # Match a token and advance if matched
        if self.current_token.type == token_type:
            self.advance()
            return True
        return False

    def parse(self) -> Program:
        # Parse a full program (list of statements)
        body = []
        self.skip_newlines()
        while self.current_token.type != 'EOF':
            stmt = self.parse_statement()
            if stmt:
                body.append(stmt)
            self.skip_newlines()
        return Program(body)

    def parse_statement(self) -> ASTNode:
        # Decide which type of statement to parse
        if self.current_token.type == 'KEYWORD':
            if self.current_token.value == 'def':
                return self.parse_function_def()
            elif self.current_token.value == 'return':
                return self.parse_return()
            elif self.current_token.value == 'if':
                return self.parse_if()
            elif self.current_token.value == 'while':
                return self.parse_while()
            elif self.current_token.value == 'for':
                return self.parse_for()

        # If not a keyword, it's either an assignment or expression
        expr = self.parse_expression()
        if isinstance(expr, Identifier) and self.match('OP_ASSIGN'):
            value = self.parse_expression()
            return Assignment(target=expr.name, value=value)
        return ExpressionStatement(expr)

    def parse_function_def(self) -> FunctionDef:
        # Parse function definition: def name(params):\n  <body>
        self.expect('KEYWORD')  # 'def'
        if self.current_token.type != 'IDENTIFIER':
            self.error("Expected function name")
        name = self.current_token.value
        self.advance()
        self.expect('LPAREN')

        # Parse parameter list
        params = []
        if self.current_token.type != 'RPAREN':
            while True:
                if self.current_token.type != 'IDENTIFIER':
                    self.error("Expected parameter name")
                params.append(self.current_token.value)
                self.advance()
                if not self.match('COMMA'):
                    break

        self.expect('RPAREN')
        self.expect('COLON')
        self.expect('NEWLINE')
        self.expect('INDENT')
        body = self.parse_block()
        return FunctionDef(name=name, params=params, body=body)

    def parse_return(self) -> Return:
        # Parse return statement
        self.expect('KEYWORD')  # 'return'
        if self.current_token.type == 'NEWLINE':
            return Return(value=None)
        value = self.parse_expression()
        return Return(value=value)

    def parse_if(self) -> IfStatement:
        # Parse if-elif-else statement
        self.expect('KEYWORD')  # 'if'
        condition = self.parse_expression()
        self.expect('COLON')
        self.expect('NEWLINE')
        self.expect('INDENT')
        then_branch = self.parse_block()

        elif_branches = []
        # Handle optional elif blocks
        while self.current_token.type == 'KEYWORD' and self.current_token.value == 'elif':
            self.advance()
            cond = self.parse_expression()
            self.expect('COLON')
            self.expect('NEWLINE')
            self.expect('INDENT')
            body = self.parse_block()
            elif_branches.append((cond, body))

        else_branch = None
        # Handle optional else block
        if self.current_token.type == 'KEYWORD' and self.current_token.value == 'else':
            self.advance()
            self.expect('COLON')
            self.expect('NEWLINE')
            self.expect('INDENT')
            else_branch = self.parse_block()

        return IfStatement(condition, then_branch, elif_branches, else_branch)

    def parse_while(self) -> WhileLoop:
        # Parse while loop
        self.expect('KEYWORD')  # 'while'
        condition = self.parse_expression()
        self.expect('COLON')
        self.expect('NEWLINE')
        self.expect('INDENT')
        body = self.parse_block()
        return WhileLoop(condition, body)

    def parse_for(self) -> ForLoop:
        # Parse for loop: for x in iterable:
        self.expect('KEYWORD')  # 'for'
        if self.current_token.type != 'IDENTIFIER':
            self.error("Expected variable name")
        var = self.current_token.value
        self.advance()
        self.expect('KEYWORD')  # 'in'
        iterable = self.parse_expression()
        self.expect('COLON')
        self.expect('NEWLINE')
        self.expect('INDENT')
        body = self.parse_block()
        return ForLoop(var, iterable, body)

    def parse_block(self) -> List[ASTNode]:
        # Parse an indented block of statements
        body = []
        self.skip_newlines()
        while self.current_token.type not in ('DEDENT', 'EOF'):
            stmt = self.parse_statement()
            if stmt:
                body.append(stmt)
            self.skip_newlines()
        self.expect('DEDENT')
        return body

    def parse_expression(self, precedence=0) -> ASTNode:
        # Parse binary expressions with precedence (e.g., 1 + 2 * 3)
        left = self.parse_primary()
        while self.is_operator(self.current_token) and self.get_precedence(self.current_token) >= precedence:
            op_token = self.current_token
            self.advance()
            right = self.parse_expression(self.get_precedence(op_token) + 1)
            left = BinaryOp(left=left, operator=op_token.value, right=right)
        return left

    def parse_primary(self) -> ASTNode:
        # Parse literals, variables, function calls, unary ops, and parenthesis
        token = self.current_token
        if token.type == 'INTEGER':
            self.advance()
            return Literal(value=int(token.value))
        elif token.type == 'FLOAT':
            self.advance()
            return Literal(value=float(token.value))
        elif token.type == 'STRING':
            self.advance()
            return Literal(value=token.value[1:-1])  # Remove surrounding quotes
        elif token.type == 'KEYWORD':
            if token.value in ('True', 'False', 'None'):
                self.advance()
                val = {'True': True, 'False': False, 'None': None}[token.value]
                return Literal(val)
        elif token.type == 'IDENTIFIER':
            self.advance()
            if self.match('LPAREN'):  # Function call
                args = []
                if self.current_token.type != 'RPAREN':
                    while True:
                        args.append(self.parse_expression())
                        if not self.match('COMMA'):
                            break
                self.expect('RPAREN')
                return Call(func=Identifier(token.value), args=args)
            return Identifier(name=token.value)
        elif token.type == 'OP_MINUS':
            self.advance()
            operand = self.parse_primary()
            return UnaryOp(operator='-', operand=operand)
        elif token.type == 'LPAREN':
            self.advance()
            expr = self.parse_expression()
            self.expect('RPAREN')
            return expr
        else:
            self.error(f"Unexpected token {token}")

    def is_operator(self, token: Token) -> bool:
        # Check if token is an operator
        return token.type.startswith('OP_')

    def get_precedence(self, token: Token) -> int:
        # Define operator precedence levels (higher = tighter binding)
        precedence = {
            'OP_OR': 1,
            'OP_AND': 2,
            'OP_EQ': 3, 'OP_NE': 3,
            'OP_LT': 4, 'OP_LE': 4, 'OP_GT': 4, 'OP_GE': 4,
            'OP_PLUS': 5, 'OP_MINUS': 5,
            'OP_MULT': 6, 'OP_MOD': 6,
            'OP_ASSIGN': 0,  # Assignment is handled separately
        }
        return precedence.get(token.type, -1)


## Testing upto parser and AST

In [29]:

# code = '''
# def add(x, y):
#     return x + y

# a = add(3, 5)
# '''

# code = """
# def foo(x):
#     if x > 0:
#         return x
#     else:
#         return -x
# """

code = '''
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)

def is_even(num):
    return num % 2 == 0

x = 5
fact = factorial(x)

if is_even(fact):
    print("Even factorial")
else:
    print("Odd factorial")

for i in range(3):
    result = factorial(i)
    print(result)
'''


lexer = Lexer(code)
parser = Parser(lexer)
ast = parser.parse()

#Console view
print_ast(ast)


#graph
dot = visualize_ast(ast)
dot.render('ast_tree', view=True, format='png')




Program
  FunctionDef(factorial)
    IfStatement
      Condition:
        BinaryOp(==)
          Identifier(n)
          Literal(0)
      Then:
        Return
          Literal(1)
      Else:
        Return
          BinaryOp(*)
            Identifier(n)
            Call
              Identifier(factorial)
                BinaryOp(-)
                  Identifier(n)
                  Literal(1)
  FunctionDef(is_even)
    Return
      BinaryOp(==)
        BinaryOp(%)
          Identifier(num)
          Literal(2)
        Literal(0)
  ExpressionStatement
    BinaryOp(=)
      Identifier(x)
      Literal(5)
  ExpressionStatement
    BinaryOp(=)
      Identifier(fact)
      Call
        Identifier(factorial)
          Identifier(x)
  IfStatement
    Condition:
      Call
        Identifier(is_even)
          Identifier(fact)
    Then:
      ExpressionStatement
        Call
          Identifier(print)
            Literal('Even factorial')
    Else:
      ExpressionStatement
        Call
    

'ast_tree.png'